In [2]:
import io
import sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder.appName('PySparkJob').getOrCreate()

In [4]:
spark

In [5]:
spark.conf.set('spark.sql.session.timeZone', 'GMT+3')

In [6]:
df = spark.read.parquet('_data/clickstream.parquet')

In [7]:
df.show(2)

+----------+-------------------+-----+--------+------+---------------+-----------------+------------+-------+---------+---------------------+
|      date|               time|event|platform| ad_id|client_union_id|compaign_union_id|ad_cost_type|ad_cost|has_video|target_audience_count|
+----------+-------------------+-----+--------+------+---------------+-----------------+------------+-------+---------+---------------------+
|2019-04-01|2019-04-01 00:00:48| view| android| 45061|          34734|            45061|         CPM|  200.6|        0|              1955269|
|2019-04-01|2019-04-01 00:00:48| view|     web|121288|         121288|           121288|         CPM|  187.4|        0|               232011|
+----------+-------------------+-----+--------+------+---------------+-----------------+------------+-------+---------+---------------------+
only showing top 2 rows



In [8]:
df.columns

['date',
 'time',
 'event',
 'platform',
 'ad_id',
 'client_union_id',
 'compaign_union_id',
 'ad_cost_type',
 'ad_cost',
 'has_video',
 'target_audience_count']

In [9]:
df.count()

1000000

In [10]:
ad_cnt = df[['ad_id']].distinct().count()
ad_cnt

965

In [11]:
def get_features(df):
    ndf = df.withColumn('is_cpm', F.when(col('ad_cost_type') =='CPM', 1).otherwise(0)) \
            .withColumn('is_cpc', F.when(col('ad_cost_type') =='CPC', 1).otherwise(0)) \
            .withColumn('is_view', F.when(col('event') =='view', 1).otherwise(0)) \
            .withColumn('is_click', F.when(col('event') =='click', 1).otherwise(0))
    
    result_df = ndf.groupBy('ad_id') \
        .agg(F.max(col('target_audience_count')), 
             F.max(col('has_video')), 
             F.max(col('is_cpm')), 
             F.max(col('is_cpc')),
             F.max(col('ad_cost')),
             F.sum(col('is_view')),
             F.sum(col('is_click')),
             F.countDistinct(col('date')).astype('int'))\
        .withColumnRenamed('max(target_audience_count)', 'target_audience_count') \
        .withColumnRenamed('max(has_video)', 'has_video') \
        .withColumnRenamed('max(is_cpm)', 'is_cpm') \
        .withColumnRenamed('max(is_cpc)', 'is_cpc') \
        .withColumnRenamed('max(ad_cost)', 'ad_cost') \
        .withColumnRenamed('sum(is_view)', 'views_cnt') \
        .withColumnRenamed('sum(is_click)', 'cliks_cnt') \
        .withColumnRenamed('CAST(count(DISTINCT date) AS INT)', 'day_count') \
        .withColumn('CTR', col('cliks_cnt') / col('views_cnt')) \
        .drop(col('views_cnt')) \
        .drop(col('cliks_cnt')) \
        .sort('ad_id', ascending=True)
        
    
    return result_df

In [12]:
result_df = get_features(df)
result_df.show(5)

+-----+---------------------+---------+------+------+-------+---------+--------------------+
|ad_id|target_audience_count|has_video|is_cpm|is_cpc|ad_cost|day_count|                 CTR|
+-----+---------------------+---------+------+------+-------+---------+--------------------+
|    2|                14841|        0|     1|     0|  186.4|        2|0.009900990099009901|
|    3|                45035|        0|     0|     1|   46.7|        2|0.016304347826086956|
| 1902|                 1970|        0|     0|     1|   42.5|        2|0.024390243902439025|
| 2064|              4145879|        0|     1|     0|  203.4|        2|                 0.0|
| 2132|               672432|        0|     0|     1|   47.5|        3|0.017241379310344827|
+-----+---------------------+---------+------+------+-------+---------+--------------------+
only showing top 5 rows



In [13]:
result_df.dtypes

[('ad_id', 'int'),
 ('target_audience_count', 'decimal(10,0)'),
 ('has_video', 'int'),
 ('is_cpm', 'int'),
 ('is_cpc', 'int'),
 ('ad_cost', 'double'),
 ('day_count', 'int'),
 ('CTR', 'double')]

In [14]:
result_cnt = result_df.count()
result_cnt

965

In [15]:
result_cnt == ad_cnt

True

In [16]:
train, test, validate = result_df.randomSplit([0.5, 0.25, 0.25])

In [17]:
train_cnt = train.count()
test_cnt = test.count()
validate_cnt = validate.count()

train_cnt, test_cnt, validate_cnt

(469, 240, 256)

In [18]:
train_cnt + test_cnt + validate_cnt == result_cnt

True